## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jason\anaconda3\envs\PythonData\lib\site-packages

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("../Resources/charity_data.csv").set_index('EIN')
df.head(1)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(columns=['NAME', 'STATUS'], inplace=True)

In [4]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
vc = df['APPLICATION_TYPE'].value_counts()
vc

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
repl = vc.loc[lambda x: x<= vc[8]].reset_index()['index']
repl

0     T9
1    T13
2    T12
3     T2
4    T14
5    T25
6    T29
7    T15
8    T17
Name: index, dtype: object

In [7]:
# Replace in dataframe
for app in repl:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
vc = df['CLASSIFICATION'].value_counts()
vc[0:11]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
repl = vc.loc[lambda x: x< vc[7]].reset_index()['index']
repl

0     C5000
1     C1270
2     C2700
3     C2800
4     C7100
      ...  
58    C2150
59    C6100
60    C1570
61    C1236
62    C2500
Name: index, Length: 63, dtype: object

In [10]:
# Replace in dataframe
for app in repl:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dums = pd.get_dummies(df)
dums.shape

(34299, 46)

In [12]:
# Split our preprocessed data into our features and target arrays
y = dums['IS_SUCCESSFUL']
X = dums.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layers = [
    {'units' : 100, 'activation': 'relu', 'input_dim': X_train_scaled.shape[1]},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 1, 'activation': 'sigmoid'}
]

In [15]:

nn = tf.keras.models.Sequential()

for layer in layers:
    nn.add(tf.keras.layers.Dense(**layer))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4600      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 14,801
Trainable params: 14,801
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5699 - acc: 0.7221
Epoch 2/200
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5546 - acc: 0.7288
Epoch 3/200
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5511 - acc: 0.7328
Epoch 4/200
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5496 - acc: 0.7320
Epoch 5/200
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5479 - acc: 0.7328
Epoch 6/200
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5473 - acc: 0.7329
Epoch 7/200
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5461 - acc: 0.7341
Epoch 8/200
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5455 - acc: 0.7355
Epoch 9/200
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5450 - acc: 0.7352
Epoch 10/200
25724/25724 [==============================] - 1s 32us/sampl

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6216 - acc: 0.7287
Loss: 0.6216265717053205, Accuracy: 0.7287463545799255


In [19]:
# Export our model to HDF5 file
nn.save('model6.h5')